### Motor Imagery

In [3]:
eeg = scipy.io.loadmat(root + 'eeg.mat')

In [5]:
x_train = np.array(eeg['x_train'])
y_train = np.array(eeg['y_train'])
x_test = np.array(eeg['x_te'])
y_test = np.array(eeg['y_te'])
ytr = np.array([x[0] for x in y_train])
yte = np.array([x[0] for x in y_test])

x_train.shape

(768, 3, 112)

In [56]:
matrix = np.full((112,255),0,dtype = complex)
for i in range(0,112):

    sample1 = librosa.stft(x_train[:,0,i], n_fft = 64, hop_length=48, window='blackman')
    sample2 = librosa.stft(x_train[:,1,i], n_fft = 64, hop_length=48, window='blackman')
    sample3 = librosa.stft(x_train[:,2,i], n_fft = 64, hop_length=48, window='blackman')
    matrix[i]= (np.hstack((sample1[2:7, :].reshape(1, 85),sample2[2:7, :].reshape(1, 85),sample3[2:7, :].reshape(1, 85))))
# matrix.shape
matrix = np.transpose(matrix)
matrix.shape

(255, 112)

In [57]:
matrix_test = np.full((28,255),0,dtype = complex)
for i in range(0,28):

    sample1 = librosa.stft(x_test[:,0,i], n_fft = 64, hop_length=48, window='blackman')
    sample2 = librosa.stft(x_test[:,1,i], n_fft = 64, hop_length=48, window='blackman')
    sample3 = librosa.stft(x_test[:,2,i], n_fft = 64, hop_length=48, window='blackman')
    matrix_test[i]= (np.hstack((sample1[2:7, :].reshape(1, 85),sample2[2:7, :].reshape(1, 85),sample3[2:7, :].reshape(1, 85))))
# matrix.shape
matrix_test = np.transpose(matrix_test)
matrix_test.shape

(255, 28)

In [58]:
def create_WH(matrix,rank):
    W = np.ones((matrix.shape[0],rank))
    H = np.ones((rank,matrix.shape[1]))

    return W,H

In [59]:
w1,h1 = create_WH(matrix,2)
w2,h2 = create_WH(matrix,4)
w3,h3 = create_WH(matrix,6)
w4,h4 = create_WH(matrix,8)
w5,h5 = create_WH(matrix,10)
w6,h6 = create_WH(matrix,15)
w7,h7 = create_WH(matrix,20)




In [60]:
def find_NMF(matrix,w,h,lr):
    prev_obj = np.full((matrix.shape),np.inf)


    for i in range(0,5000):
        o = 0.5*np.trace(np.transpose(matrix - np.matmul(w,h)))
        o*= (matrix - np.matmul(w,h))
        prev_w = copy.deepcopy(w)
        prev_h = copy.deepcopy(h)
        w -= lr*(np.matmul(w,np.matmul(h,np.transpose(h))) - np.matmul(matrix,np.transpose(h)))
        h -= lr*(np.matmul(np.matmul(np.transpose(w),w),h) - np.matmul(np.transpose(w),matrix))
        if i==1000: lr/=10
        if np.allclose(prev_obj,o,atol=1e-3,rtol=1e-3):
            print("Converged at ", i)
            break
        prev_obj = copy.deepcopy(o)

    return w,h

In [61]:
w1,h1 = find_NMF(np.abs(matrix),w1,h1,1e-05)
w2,h2 = find_NMF(np.abs(matrix),w2,h2,1e-05)
w3,h3 = find_NMF(np.abs(matrix),w3,h3,1e-05)
w4,h4 = find_NMF(np.abs(matrix),w4,h4,1e-05)
w5,h5 = find_NMF(np.abs(matrix),w5,h5,1e-05)
w6,h6 = find_NMF(np.abs(matrix),w6,h6,1e-05)
w7,h7 = find_NMF(np.abs(matrix),w7,h7,1e-05)


Converged at  1002
Converged at  946
Converged at  739
Converged at  610
Converged at  523
Converged at  390
Converged at  315


In [62]:
h1.shape,h2.shape

((2, 112), (4, 112))

In [63]:
wt1,ht1 = create_WH(matrix_test,2)
wt2,ht2 = create_WH(matrix_test,4)
wt3,ht3 = create_WH(matrix_test,6)
wt4,ht4 = create_WH(matrix_test,8)
wt5,ht5 = create_WH(matrix_test,10)
wt6,ht6 = create_WH(matrix_test,15)
wt7,ht7 = create_WH(matrix_test,20)
wt1.shape,ht1.shape

((255, 2), (2, 28))

In [64]:
wt1,ht1 = find_NMF(np.abs(matrix_test),wt1,ht1,1e-05)
wt2,ht2 = find_NMF(np.abs(matrix_test),wt2,ht2,1e-05)
wt3,ht3 = find_NMF(np.abs(matrix_test),wt3,ht3,1e-05)
wt4,ht4 = find_NMF(np.abs(matrix_test),wt4,ht4,1e-05)
wt5,ht5 = find_NMF(np.abs(matrix_test),wt5,ht5,1e-05)
wt6,ht6 = find_NMF(np.abs(matrix_test),wt6,ht6,1e-05)
wt7,ht7 = find_NMF(np.abs(matrix_test),wt7,ht7,1e-05)


Converged at  681
Converged at  514
Converged at  408
Converged at  340
Converged at  293
Converged at  219
Converged at  177


In [65]:
def test_knn(train, test, k, train_labels):
    h = []
    dist, label = np.sqrt(np.power((train - test), 2)), train_labels
    d = dict(zip(dist, label))
    heappush(h, (dist, label))
    smallest = heapq.nsmallest(k, dist)
    return [d[x] for x in smallest]

In [66]:
neighbours = [5,10,15,20]
def get_preds(train,test,ytr):
    all_preds = []
    for k in neighbours:
        train_preds = []

        for i in range(0,test.shape[1]):
            d= {}
            result = test_knn(train[0],test[0][i],k,ytr)
            for j in result:
                if j not in d:
                    d[j]=1
                else:
                    d[j]+=1
            train_preds.append(max(d, key=d.get))
        all_preds.append(train_preds)
    return np.array(all_preds)


In [67]:
pred_train1,pred_test1 = get_preds(h1,h1,ytr),get_preds(h1,ht1,ytr)
pred_train2,pred_test2 = get_preds(h2,h2,ytr),get_preds(h2,ht2,ytr)
pred_train3,pred_test3 = get_preds(h3,h3,ytr),get_preds(h3,ht3,ytr)
pred_train4,pred_test4 = get_preds(h4,h4,ytr),get_preds(h4,ht4,ytr)
pred_train5,pred_test5 = get_preds(h5,h5,ytr),get_preds(h5,ht5,ytr)
pred_train6,pred_test6 = get_preds(h6,h6,ytr),get_preds(h6,ht6,ytr)
pred_train7,pred_test7 = get_preds(h7,h7,ytr),get_preds(h7,ht7,ytr)

In [68]:
pred_train1.shape,pred_test1.shape,ytr.shape,yte.shape

((4, 112), (4, 28), (112,), (28,))

In [69]:
def acc(preds, gt):
    return round(np.mean(preds == gt) * 100, 2)

In [70]:
acc_list = []

a1 = [acc(row, ytr) for row in pred_train1]
ta1 = [acc(row, yte) for row in pred_test1]

a2 = [acc(row, ytr) for row in pred_train2]
ta2 = [acc(row, yte) for row in pred_test2]

a3 = [acc(row, ytr) for row in pred_train3]
ta3 = [acc(row, yte) for row in pred_test3]

a4 = [acc(row, ytr) for row in pred_train4]
ta4 = [acc(row, yte) for row in pred_test4]

a5 = [acc(row, ytr) for row in pred_train5]
ta5 = [acc(row, yte) for row in pred_test5]

a6 = [acc(row, ytr) for row in pred_train6]
ta6 = [acc(row, yte) for row in pred_test6]

a7 = [acc(row, ytr) for row in pred_train7]
ta7 = [acc(row, yte) for row in pred_test7]


acc_list.append(a1)
acc_list.append(ta1)

acc_list.append(a2)
acc_list.append(ta2)

acc_list.append(a3)
acc_list.append(ta3)

acc_list.append(a4)
acc_list.append(ta4)

acc_list.append(a5)
acc_list.append(ta5)

acc_list.append(a6)
acc_list.append(ta6)

acc_list.append(a7)
acc_list.append(ta7)


acc_list = np.array(acc_list)
idx = [2,2,4,4,6,6,8,8,10,10,15,15,20,20]
typeacc = ['Train','Test']*7
my_dict = {'Accuracy type':typeacc,'Rank of H':idx,'5 Neighbours': acc_list[:,0], '10 Neighbours': acc_list[:,1], '15 Neighbours': acc_list[:,2], '20 Neighbours': acc_list[:,3]}
Accuracy_table = pd.DataFrame(my_dict)

In [71]:
Accuracy_table

,Accuracy type,Rank of H,5 Neighbours,10 Neighbours,15 Neighbours,20 Neighbours
0,Train,2,69.64,76.79,63.39,65.18
1,Test,2,53.57,50.00,50.00,53.57
2,Train,4,67.86,78.57,65.18,63.39
3,Test,4,67.86,57.14,50.00,50.00
4,Train,6,67.86,79.46,63.39,66.96
5,Test,6,67.86,60.71,53.57,50.00
6,Train,8,66.07,79.46,64.29,66.07
7,Test,8,64.29,57.14,50.00,50.00
8,Train,10,66.07,80.36,64.29,66.07
9,Test,10,64.29,60.71,50.00,50.00
